**2**. Number theory and a Google recruitment puzzle

Find the first 10-digit prime in the decimal expansion of 17π. 

The first 5 digits in the decimal expansion of π are 14159. The first 4-digit prime in the decimal expansion of π are 4159. You are asked to find the first 10-digit prime in the decimal expansion of 17π. First solve sub-problems (divide and conquer):

- Write a function to generate an arbitrary large expansion of a mathematical expression like π. Hint: You can use the standard library `decimal` or the 3rd party library `sympy` to do this
- Write a function to check if a number is prime. Hint: See Sieve of Eratosthenes
- Write a function to generate sliding windows of a specified width from a long iterable (e.g. a string representation of a number)

Write unit tests for each of these three functions. You are encouraged, but not required, to try [test-driven development](https://en.wikipedia.org/wiki/Test-driven_development).

Now use these helper functions to write the function that you need.
Write a unit test for this final function, given that the first 10-digit prime in the expansion e is 7427466391. Finally, solve the given problem.



In [38]:
import numpy as np

## Function 1 : Generate an arbitrary large expansion of a mathematical expression.

Use directly from `sympy` package to expand an mathematical expression.

And we also check if the input expand_decimal is integer or float, raise error if not.

In [263]:
from sympy import *

def expand(expression, expand_decimal):
    """Return the expression with decimal length specified"""
    
    if type(expand_decimal) not in [int, float]:
        raise TypeError("The specified digits length of expansion should be a float or an integer")
    
    return N(expression, expand_decimal)

In [65]:
expand(17*pi,5)

53.407

In [276]:
expand('17*pi',5)

53.407

## Function 2 : Check if a number is prime

The idea of this function follows Sieve of Eratosthenes. After considering some special case, such that number 2 is a prime, we can first sieve all even numbers because they are not prime, can be divided by 2. Then we can sieve multiples of odd numbers in a specific range, setting the upper bound as the square root of the number is economic way, and add 1 to be safer, and we skip the even numbers because we already sieve for 2. 

In [264]:
def check_prime(n):
    """Check if the number is a prime"""
    if n == 2:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    for i in range(3, int(np.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

In [277]:
check_prime(2)

True

In [279]:
check_prime(15)

False

## Function 3 : Generate sliding windows of a specified width from a long iterable

For each input number, create the sliding window from start position and with specified length, if there is dot included, we should split by dot and join all numbers from the int and decimal portion.

In [305]:
def sliding_window(num, start, length):
    """Return the string of the number starts from start position with specified length"""
    
    if type(start) != int or type(length) != int:
        raise TypeError("n_len and n_dec need to be integers")
    
    string = ''.join(str(num).split("."))[start : start + length]
    return string

In [307]:
sliding_window(3.15663347, 1, 4)

'1566'

In [308]:
sliding_window(0.15663347, 0, 4)

'0156'

## Unit test for 3 functions

In [349]:
%%file functions.py

from sympy import *
import numpy as np


def expand(expression, expand_decimal):
    """Return the expression with decimal length specified"""
    
    if type(expand_decimal) not in [int, float]:
        raise TypeError("The specified digits length of expansion should be a float or an integer")
    
    return N(expression, expand_decimal)

def check_prime(n):
    """Check if the number is a prime"""
    if n == 2:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    for i in range(3, int(np.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

def sliding_window(num, start, length):
    """Return the string of the number starts from start position with specified length"""
    
    if type(start) != int or type(length) != int:
        raise TypeError("n_len and n_dec need to be integers")
    
    string = ''.join(str(num).split("."))[start : start + length]
    return string

Overwriting functions.py


In [350]:
%%file test_functions.py

import math
import unittest
from functions import expand, check_prime, sliding_window


class TestHelpers(unittest.TestCase):
    def test_expand(self):
        self.assertEqual(str(expand(math.pi, 4)), format(math.pi, '.3f'))
        self.assertEqual(str(expand(math.e, 30)), format(math.e, '.29f'))
        self.assertEqual(str(expand(7.6*math.e, 8)), format(7.6*math.e, '.6f'))
    
    def test_expand_input(self):
        self.assertRaises(TypeError, expand, "5")
    
    def test_prime(self):
        self.assertTrue(check_prime(2))
        self.assertTrue(check_prime(5))
        self.assertTrue(check_prime(7427466391))
        self.assertFalse(check_prime(16))
        
    def test_gen(self):
        self.assertEqual(sliding_window(3.1415926589, 4, 3), '592')
        self.assertEqual(sliding_window(math.e, 6, 8), format(math.e, '.20f')[7:7+8])
    
if __name__ == '__main__':
    unittest.main()

Overwriting test_functions.py


In [372]:
! python3 -m unittest test_functions.py

....
----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK


## Find first n-digit prime in the decimal expansion of a number

1. For each expression, we first expand it to specified decimals, if it is not specified, we use default 1000.

2. We create sliding windows for the certain number of digit in the expanded expression. It should be a loop because we check from the beginning of the expanded expression and add 1 to the start position each time (shift the window to the right for 1 position each time).

3. Check if the number has length of the prime length we want, because if the number has 0 at the beginning, the length is shortened by 1, thus we set this as the one of the checking condition. And another checking condition is if the nubmer in sliding window is prime or not, and we only need the first prime.

In [309]:
def prime_in_num(expression, prime_dig, expand_decimal = 1000):
    """Return the first prime of specified length in an expression"""
    
    expanded = expand(expression, expand_decimal)
    
    for i in range(expand_decimal):
        num_int = int(sliding_window(expanded, i, prime_dig))
        
        if (len(str(num_int)) == prime_dig) and (check_prime(num_int)):
            return num_int
        else: 
            i += 1
                
        if i > expand_decimal - prime_dig:
            return("There is no prime with specified length within the decimal expansion of the expression")

Check if the return message works.

In [352]:
prime_in_num(E, 10, 20)

'There is no prime with specified length within the decimal expansion of the expression'

Check with e, which should have 10-digit prime 7427466391 in decimal expansion.

In [353]:
prime_in_num(E, 10)

7427466391

## Unit test for main function

In [364]:
%%file main_function.py

from functions import expand, check_prime, sliding_window


def prime_in_num(expression, prime_dig, expand_decimal = 1000):
    """Return the first prime of specified length in an expression"""
    
    expanded = expand(expression, expand_decimal)
    
    for i in range(expand_decimal):
        num_int = int(sliding_window(expanded, i, prime_dig))
        
        if (len(str(num_int)) == prime_dig) and (check_prime(num_int)):
            return num_int
        else: 
            i += 1
                
        if i > expand_decimal - prime_dig:
            return("There is no prime with specified length within the decimal expansion of the expression")

Overwriting main_function.py


In [369]:
%%file test_main_function.py

from sympy import *
import math
import unittest
from functions import expand, check_prime, sliding_window
from main_function import prime_in_num

class TestHelpers(unittest.TestCase):
    def test_main(self):
        self.assertEqual(prime_in_num(E, 10), 7427466391)
        self.assertEqual(prime_in_num(math.pi, 5), 14159)
        self.assertEqual(prime_in_num(math.pi, 4), 4159)
    
if __name__ == '__main__':
    unittest.main()

Overwriting test_main_function.py


In [370]:
! python3 -m unittest test_main_function.py

.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


## Solve the problem

Find 10-digit prime in decimal expansion of $17*\pi$

In [371]:
prime_in_num(17*pi, 10)

8649375157